In [56]:
# Experiment Settings, logger, plotter
from utils.config import get_config
from utils.logger import Logger
from utils.plotter import MetricsPlotter
from utils.utils import set_settings, set_seed
config = get_config()

In [57]:
config.__dict__['rank'] = 1
config

TestConfig(classification=True, Ablation=0, bs=128, lr=0.001, decay=0.0001, loss_func='L1Loss', optim='AdamW', path='./datasets/', dataset='cpu', train_size=500, density=0.8, eval_set=False, logger='None', model='ours', rank=1, retrain=True, seed=0, rounds=5, epochs=250, patience=45, verbose=1, device='mps', debug=True, experiment=False, program_test=False, record=True)

In [58]:
command = f"python train_model.py --config_path ./exper_config.py --exp_name TestConfig " \
              f"--density 1 --retrain 1 --rounds 1  --dataset food --eval_set 1"

In [59]:
command

'python train_model.py --config_path ./exper_config.py --exp_name TestConfig --density 1 --retrain 1 --rounds 1  --dataset food --eval_set 1'

In [81]:
def add_parameter(command: str, params: dict) -> str:
    for param_name, param_value in params.items():
        command += f" --{param_name} {param_value}"
    return command

# Original string
commands = ['python train_model.py --config_path ./exper_config.py --exp_name TestConfig --density 1 --retrain 1 --rounds 1  --dataset food --eval_set 1']

# Dictionary with parameters to add
params = {'rank': 10}


In [82]:
hyper = {'rank' :10}
hyper.keys()

dict_keys(['rank'])

In [83]:
if hyper is not None:
    commands = [add_parameter(command, hyper) for command in commands]

In [84]:
commands

['python train_model.py --config_path ./exper_config.py --exp_name TestConfig --density 1 --retrain 1 --rounds 1  --dataset food --eval_set 1 --rank 10']